In [1]:
library(IRdisplay)
display_html("<style>.container { width:100% !important; }</style>")

.libPaths()

[1] "/hps/software/users/marioni/Leah/miniconda3/envs/basic_renv/lib/R/library"

In [2]:
library(Seurat)
library(Matrix)
library(parallel)

Attaching SeuratObject

Attaching sp



In [3]:
srat <- readRDS('/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/seurat_final.rds')

In [4]:
NNgraph_idx <- split(as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@i+1,
                     rownames(srat@graphs[['RNA_nn']])[as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@j+1]
                     )
saveRDS(NNgraph_idx, 'NNgraph_idx.RDS', compress = TRUE)

An object of class Seurat 
30406 features across 77683 samples within 1 assay 
Active assay: RNA (30406 features, 4214 variable features)
 2 dimensional reductions calculated: pca, umap

In [ ]:
counts <- t(srat@assays$RNA@counts)
saveRDS(counts, 'counts.RDS', compress = TRUE)

genemeta <- as.data.table(srat@assays$RNA@meta.features)
fwrite(genemeta, "genemeta.txt.gz")

normed <- srat@assays$RNA@data
saveRDS(normed, 'normed.RDS', compress = TRUE)

In [5]:
parallel.function <- function(ens_id) {
    to.plot <- as.matrix(srat@assays[['RNA']]@data[c(ens_id),])
    idx <- split(as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@i+1, rownames(srat@graphs[['RNA_nn']])[as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@j+1])
    gene_ave <- lapply(idx, function(x) mean(to.plot[x]))
    gene_ave <- do.call(rbind, gene_ave)
}

In [8]:
gene_aves <- mclapply(rownames(srat), FUN=parallel.function)

In [9]:
out <- do.call(cbind, gene_aves)
colnames(out) <- rownames(srat)
out <- out[,colSums(out) != 0]
saveRDS(out, 'out.RDS')

In [11]:
as(out, "dgTMatrix")

In [ ]:
writeMM(as(out, "dgTMatrix"), '/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/smoothed_mat.mtx.gz')

In [5]:
out <- readRDS('out.RDS')
colnames(out) <- rownames(srat)
out <- out[,colSums(out) != 0]

In [7]:
library(data.table)

In [8]:
fwrite(out, '/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/out.txt.gz')

x being coerced from class: matrix to data.table



In [14]:
counts_c <- srat@assays$RNA@counts
counts_r <- t(counts_c)

In [21]:
start <- Sys.time()
tmp <- counts_c['ENSMUSG00000062327',]
end <- Sys.time()
end-start

start <- Sys.time()
tmp <- counts_r[,'ENSMUSG00000062327']
end <- Sys.time()
end-start

Time difference of 3.292553 secs

Time difference of 0.9617112 secs

In [6]:
object.size(out)
object.size(srat)
object.size(srat@assays$RNA@counts)
object.size(srat@assays$RNA@data)
out_dgR <- as(out, "dgRMatrix")
object.size(out_dgR)

18903070752 bytes

9512855392 bytes

4599893488 bytes

4599893488 bytes

11523201712 bytes

In [7]:
out_dgR_signif <- signif(out_dgR, 3)
dim(out_dgR_signif)

[1] 77683 30406

In [21]:
out <- out[,colSums(out) != 0]

In [ ]:
out_dgC <- as(out, "dgCMatrix")
object.size(out_dgC)
writeMM(as(out, "dgCMatrix"),
        '/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/smoothed_mat.mtx.gz'
       )

11525234176 bytes

In [7]:
out_dgC <- as(out, "dgCMatrix")
object.size(out_dgC)

out_TdgC <- as(t(out), "dgCMatrix")
object.size(out_TdgC)

out_dgT <- as(t(out), "dgTMatrix")
object.size(out_dgT)

11523012608 bytes

11523201712 bytes

15361575480 bytes

In [ ]:
out_dgR <- as(out, "dgRMatrix")
saveRDS(out_dgR, 'out_dgR.RDS', compress = TRUE)

In [ ]:
object.size(out_dgR)

In [ ]:
out_dgC <- as(out, "dgCMatrix")
saveRDS(out_dgC, 'out_dgC.RDS')

out_TdgC <- as(t(out), "dgCMatrix")
saveRDS(out_sparse, 'out_TdgC.RDS')

In [ ]:
object.size(out_dgR)
object.size(out_dgC)
object.size(out_TdgC)

In [10]:
idx <- split(as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@i+1, rownames(srat@graphs[['RNA_nn']])[as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@j+1])
gene_ave <- lapply(idx, function(x) rowMeans(srat@assays[['RNA']]@data[,x]))
to.plot <- do.call(rbind, gene_ave)
head(to.plot)

In [12]:
srat@assays$RNA@meta.features

,ens_id,symbol,cc_gene,gastr_only_hvg
,<chr>,<chr>,<lgl>,<lgl>
ENSMUSG00000051951,ENSMUSG00000051951,Xkr4,FALSE,FALSE
ENSMUSG00000089699,ENSMUSG00000089699,Gm1992,FALSE,FALSE
ENSMUSG00000102343,ENSMUSG00000102343,Gm37381,FALSE,FALSE
ENSMUSG00000025900,ENSMUSG00000025900,Rp1,FALSE,FALSE
ENSMUSG00000025902,ENSMUSG00000025902,Sox17,FALSE,TRUE
ENSMUSG00000104328,ENSMUSG00000104328,Gm37323,FALSE,FALSE
ENSMUSG00000033845,ENSMUSG00000033845,Mrpl15,FALSE,FALSE
ENSMUSG00000025903,ENSMUSG00000025903,Lypla1,FALSE,FALSE
ENSMUSG00000033813,ENSMUSG00000033813,Tcea1,FALSE,FALSE


In [19]:
start_time <- Sys.time()
to.plot <- as.matrix(srat@assays[['RNA']]@data[c('ENSMUSG00000051951'),])
idx <- split(as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@i+1, rownames(srat@graphs[['RNA_nn']])[as(t(srat@graphs[['RNA_nn']]), "dgTMatrix")@j+1])
gene_ave <- lapply(idx, function(x) mean(to.plot[x]))
gene_ave <- do.call(rbind, gene_ave)
                   end_time <- Sys.time()
                   end_time - start_time

Time difference of 2.5929 secs

In [24]:
library(parallel)

In [26]:
gene_aves <- mclapply(rownames(srat), FUN=parallel.function)

[1] "ENSMUSG00000051951" "ENSMUSG00000089699" "ENSMUSG00000102343"
    [4] "ENSMUSG00000025900" "ENSMUSG00000025902" "ENSMUSG00000104328"
    [7] "ENSMUSG00000033845" "ENSMUSG00000025903" "ENSMUSG00000033813"
   [10] "ENSMUSG00000002459" "ENSMUSG00000085623" "ENSMUSG00000033793"
   [13] "ENSMUSG00000025905" "ENSMUSG00000033774" "ENSMUSG00000025907"
   [16] "ENSMUSG00000090031" "ENSMUSG00000087247" "ENSMUSG00000033740"
   [19] "ENSMUSG00000051285" "ENSMUSG00000097797" "ENSMUSG00000103067"
   [22] "ENSMUSG00000025909" "ENSMUSG00000061024" "ENSMUSG00000025911"
   [25] "ENSMUSG00000067879" "ENSMUSG00000099827" "ENSMUSG00000025912"
   [28] "ENSMUSG00000045210" "ENSMUSG00000097893" "ENSMUSG00000025915"
   [31] "ENSMUSG00000046101" "ENSMUSG00000098234" "ENSMUSG00000099032"
   [34] "ENSMUSG00000025916" "ENSMUSG00000087199" "ENSMUSG00000025917"
   [37] "ENSMUSG00000056763" "ENSMUSG00000067851" "ENSMUSG00000042501"
   [40] "ENSMUSG00000048960" "ENSMUSG00000057715" "ENSMUSG00000097171"
   [43] "ENSMUSG00000101314" "ENSMUSG00000016918" "ENSMUSG00000025938"
   [46] "ENSMUSG00000099498" "ENSMUSG00000042414" "ENSMUSG00000005886"
   [49] "ENSMUSG00000101476" "ENSMUSG00000025935" "ENSMUSG00000025937"
   [52] "ENSMUSG00000067813" "ENSMUSG00000025932" "ENSMUSG00000054493"
   [55] "ENSMUSG00000025930" "ENSMUSG00000032769" "ENSMUSG00000092083"
   [58] "ENSMUSG00000025925" "ENSMUSG00000032719" "ENSMUSG00000067795"
   [61] "ENSMUSG00000043716" "ENSMUSG00000025921" "ENSMUSG00000100868"
   [64] "ENSMUSG00000025920" "ENSMUSG00000089982" "ENSMUSG00000025939"
   [67] "ENSMUSG00000079658" "ENSMUSG00000097744" "ENSMUSG00000025940"
   [70] "ENSMUSG00000025779" "ENSMUSG00000101640" "ENSMUSG00000042686"
   [73] "ENSMUSG00000100110" "ENSMUSG00000025777" "ENSMUSG00000099899"
   [76] "ENSMUSG00000067780" "ENSMUSG00000100053" "ENSMUSG00000085125"
   [79] "ENSMUSG00000025776" "ENSMUSG00000099895" "ENSMUSG00000025774"
   [82] "ENSMUSG00000073735" "ENSMUSG00000067773" "ENSMUSG00000089787"
   [85] "ENSMUSG00000042596" "ENSMUSG00000025927" "ENSMUSG00000100538"
   [88] "ENSMUSG00000043760" "ENSMUSG00000089914" "ENSMUSG00000099906"
   [91] "ENSMUSG00000025929" "ENSMUSG00000041872" "ENSMUSG00000041859"
   [94] "ENSMUSG00000102121" "ENSMUSG00000097934" "ENSMUSG00000025931"
   [97] "ENSMUSG00000041809" "ENSMUSG00000041779" "ENSMUSG00000099971"
  [100] "ENSMUSG00000025933" "ENSMUSG00000025934" "ENSMUSG00000099343"
  [103] "ENSMUSG00000067750" "ENSMUSG00000041722" "ENSMUSG00000085079"
  [106] "ENSMUSG00000028033" "ENSMUSG00000041670" "ENSMUSG00000073730"
  [109] "ENSMUSG00000097970" "ENSMUSG00000100775" "ENSMUSG00000026158"
  [112] "ENSMUSG00000099503" "ENSMUSG00000026156" "ENSMUSG00000026155"
  [115] "ENSMUSG00000026154" "ENSMUSG00000026153" "ENSMUSG00000026147"
  [118] "ENSMUSG00000026141" "ENSMUSG00000073725" "ENSMUSG00000100483"
  [121] "ENSMUSG00000099392" "ENSMUSG00000033569" "ENSMUSG00000073722"
  [124] "ENSMUSG00000101514" "ENSMUSG00000048411" "ENSMUSG00000053185"
  [127] "ENSMUSG00000048874" "ENSMUSG00000026064" "ENSMUSG00000100237"
  [130] "ENSMUSG00000086727" "ENSMUSG00000100910" "ENSMUSG00000050217"
  [133] "ENSMUSG00000026063" "ENSMUSG00000026058" "ENSMUSG00000091318"
  [136] "ENSMUSG00000102917" "ENSMUSG00000104230" "ENSMUSG00000026134"
  [139] "ENSMUSG00000004768" "ENSMUSG00000042215" "ENSMUSG00000042197"
  [142] "ENSMUSG00000042182" "ENSMUSG00000026131" "ENSMUSG00000104397"
  [145] "ENSMUSG00000102336" "ENSMUSG00000042111" "ENSMUSG00000101711"
  [148] "ENSMUSG00000026127" "ENSMUSG00000026126" "ENSMUSG00000101223"
  [151] "ENSMUSG00000026125" "ENSMUSG00000026124" "ENSMUSG00000037529"
  [154] "ENSMUSG00000045815" "ENSMUSG00000045174" "ENSMUSG00000037509"
  [157] "ENSMUSG00000104002" "ENSMUSG00000037503" "ENSMUSG00000026123"
  [160] "ENSMUSG00000104417" "ENSMUSG00000101714" "ENSMUSG00000102531"
  [163] "ENSMUSG00000102279" "ENSMUSG00000103708" "ENSMUSG00000104201"
  [166] "ENSMUSG00000045216" "ENSMUSG00000104523" "ENSMUSG00000102202"
  [169] "ENSMUSG00000037470"